# Train RL Agent

Trains both the network and the agent using the actual Minecraft environment

In [6]:
from keras.models import Model
from keras.layers import Dense, Dropout, Flatten, Input, Lambda, Conv2D
from keras.optimizers import SGD, Adam
from keras import initializers
from keras import regularizers
import keras.backend as K

from anyrl.rollouts import replay

import time
import numpy as np

import minerl
import gym

import tensorflow as tf
import pickle
import collections
from collections import deque

import random

import sys
sys.path.append("./functions")

from ActionCombos import load_combos, get_all_action_combos, int_action_to_dict, convert_match_actions, match_batch_actions
from Utils import load_data, save_data
from DQfDModel import build_model, train_network

In [7]:
resources_folder = 'training-resources'
models_path = 'app/models'
environment = 'Treechop'

# Training

In [9]:
#load in all necessary data

replay_file = "../{0}/buffer_{1}_post_train.sav".format(resources_folder, environment)
model_path = '../{0}/MineRL{1}-v0_model.h5'.format(models_path, environment)
unique_angles_file = '../{0}/unique_angles_{1}.sav'.format(resources_folder, environment.lower())
combos_file = '../{0}/action_combos_{1}.sav'.format(resources_folder, environment.lower())
unique_actions_file = "../{0}/unique_actions_{1}.sav".format(resources_folder,environment.lower())
action_combos = load_data(combos_file)
unique_angles = load_data(unique_angles_file)

minerl_environment = 'MineRL{0}-v0'.format(environment)
n_action = len(action_combos)

unique_actions = load_data(unique_actions_file)

In [ ]:
#start up MineRL env
env = gym.make(minerl_environment)

In [ ]:
#load in axpert buffer and raw replay buffer
expert_buffer = replay.PrioritizedReplayBuffer(500000, alpha=0.4, beta=0.6, epsilon=0.001)
expert_buffer = pickle.load(open(replay_file, 'rb'))
replay_buffer = replay.PrioritizedReplayBuffer(500000, alpha=0.4, beta=0.6, epsilon=0.001)

In [ ]:
#load target model model
tgt_model = build_model(n_action)
tgt_model.load_weights(model_path)

In [ ]:
#load in model being trained 
model = build_model(n_action)
model.load_weights(model_path)

In [ ]:
#train 
train_network(env, model, tgt_model,expert_buffer,replay_buffer, 
              n_action, action_combos, unique_angles, action_keys, max_timesteps=750000)